In [1]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome


In [3]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]


In [4]:
def editGenDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])


In [5]:
p = 'GCGTATGC'
t = 'TATTGGCTATACGGTT'

print(editGenDistance(p, t))

2


In [6]:
t = readGenome('chr1.GRCh38.excerpt.fasta')

In [7]:
p = 'GCTGATCGATCGTACG'
print(editGenDistance(p, t))

3


In [8]:
p = 'GATTTACCAGATTGAG'
print(editGenDistance(p, t))

2


In [58]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [59]:
from itertools import permutations

In [109]:
def findKmers(read, k):
    kmers = set()
    for i in range(len(read) - k + 1):
        kmers.add(read[i:i + k])
    return kmers

In [142]:
def findKmers(read, k):
    kmers = set()
    for i in range(len(read) - k + 1):
        kmers.add(read[i:i + k])
    return kmers

def overlap_all_pairs(reads, min_length):
    pairs = set()
    goodCounter = 0
    kmerToRead = {}
    #kmers[] = set()
    
    for read in reads:
        kmersInRead = findKmers(read, min_length)
        for kmer in kmersInRead:
            if kmer not in kmerToRead :
                kmerToRead[kmer] = set()
            kmerToRead[kmer].add(read)
    
    for read in reads:
        matched = False
        otherReads = kmerToRead[read[len(read) - min_length:]]
        for otherRead in otherReads:
            if read != otherRead:
                if overlap(read, otherRead, min_length) != 0 :
                    pairs.add((read, otherRead))
                    matched = True
        if matched:
            goodCounter += 1
    
    return pairs, goodCounter

In [143]:
reads, _ = readFastq('ERR266411_1.for_asm.fastq')

In [144]:
pairs, count = overlap_all_pairs(reads, 30)
print (len(pairs))
print(count)

904746
7161


In [118]:
test_reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlap_all_pairs(test_reads, 4)
overlap_all_pairs(test_reads, 5)

[('CGTACG', 'GTACGA'),
 ('CGTACG', 'GTACGT'),
 ('TACGTA', 'ACGTAC'),
 ('GTACGT', 'TACGTA'),
 ('ACGTAC', 'CGTACG'),
 ('GTACGA', 'TACGAT')]

In [120]:
test_reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
overlap_all_pairs(test_reads, 3)
overlap_all_pairs(test_reads, 4)

[]